<a href="https://colab.research.google.com/github/karoldem/dh/blob/master/dh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import sympy

class dh:
  def __init__(self,n):
    self.table = []
    for i in range (n):
      self.table.append({'a': sympy.symbols(r'a_' + str(i+1)),
                  'al':sympy.symbols(r'\alpha_' + str(i+1)),
                  'd': sympy.symbols(r'd_' +  str(i+1)),
                  't': sympy.symbols(r'\theta_' +  str(i+1))
                  })

  def dhm(self,i):
    return (sympy.Matrix([[sympy.cos(i['t']), -sympy.sin(i['t'])*sympy.cos(i['al']), sympy.sin(i['t'])*sympy.sin(i['al']), i['a']*sympy.cos(i['t'])],
                          [sympy.sin(i['t']), sympy.cos(i['t'])*sympy.cos(i['al']), -sympy.cos(i['t'])*sympy.sin(i['al']), i['a']*sympy.sin(i['t'])],
                          [0, sympy.sin(i['al']), sympy.cos(i['al']), i['d']],
                          [0,0,0,1]]))
    
  def makematrix (self):
    t = sympy.eye(4)
    for i in self.table:
      t = t*self.dhm(i)
    return t
  
  def jacobian (self,x):
    correct_x = sympy.Matrix(1,0,[])
    for i in x:
      correct_x = sympy.Matrix([i]).col_insert(0,correct_x)
      pass
    
    t = sympy.eye(4)
    for i in self.table:
      t = t*self.dhm(i)

    self.placement = t * sympy.Matrix([[0,0,0,1]]).T
    j = self.placement.T.jacobian( correct_x )
    j.row_del(3)

    for i in range(len( j )):
      j[i] = sympy.simplify( j[i] )
    
    return j
    
  def print_transorm_matrices(self):
    for i in range (len( self.table )):
      t = self.dhm( self.table[i] )

      for j in range (16):
        t[j] = sympy.simplify(t[j])
      
      print (r'$$')
      print (r'^' + str(i) + r'T_' +str(i+1) +  r' = ')
      print(str(sympy.latex(t)))
      print (r'$$')

    print (r'$$')
    print (r'^0 T_'+ str(len(self.table)) + r' = ')
    print(str(sympy.latex( self.makematrix() )))
    print (r'$$')

Here we define our robot constructor created dh table in field `table` with special kind of variables (google sympy library for more). To define a robot, user must replace those variables with constants (or any other type - free country).
If you happen to contribute, let us agree, that this is what makes a robot waht it is and any other method must base on it (unlike in FreeCAD projest if you know what I mean)

In [0]:
antropomorphic = dh(3) #constructor creates dh table with special kind of variables (google sympy library for more)

#here we replace those variables with constatns (al means alpha, t means theta)
antropomorphic.table[0]['a'] = 0
antropomorphic.table[0]['al'] = sympy.rad(90)

antropomorphic.table[1]['d'] = 0
antropomorphic.table[1]['al'] = 0

antropomorphic.table[2]['d'] = 0
antropomorphic.table[2]['al'] = 0

method `dhm(n)` creates matrix of transformation between nth and (n-1)th coordinate system.

In [19]:
antropomorphic.dhm(antropomorphic.table[2])

Matrix([
[cos(\theta_3), -sin(\theta_3), 0, a_3*cos(\theta_3)],
[sin(\theta_3),  cos(\theta_3), 0, a_3*sin(\theta_3)],
[            0,              0, 1,                 0],
[            0,              0, 0,                 1]])

method makematrix() returns transform matrix between zeroth system and TCP

In [20]:
antropomorphic.makematrix()

Matrix([
[-sin(\theta_2)*sin(\theta_3)*cos(\theta_1) + cos(\theta_1)*cos(\theta_2)*cos(\theta_3), -sin(\theta_2)*cos(\theta_1)*cos(\theta_3) - sin(\theta_3)*cos(\theta_1)*cos(\theta_2),  sin(\theta_1), a_2*cos(\theta_1)*cos(\theta_2) - a_3*sin(\theta_2)*sin(\theta_3)*cos(\theta_1) + a_3*cos(\theta_1)*cos(\theta_2)*cos(\theta_3)],
[-sin(\theta_1)*sin(\theta_2)*sin(\theta_3) + sin(\theta_1)*cos(\theta_2)*cos(\theta_3), -sin(\theta_1)*sin(\theta_2)*cos(\theta_3) - sin(\theta_1)*sin(\theta_3)*cos(\theta_2), -cos(\theta_1), a_2*sin(\theta_1)*cos(\theta_2) - a_3*sin(\theta_1)*sin(\theta_2)*sin(\theta_3) + a_3*sin(\theta_1)*cos(\theta_2)*cos(\theta_3)],
[                             sin(\theta_2)*cos(\theta_3) + sin(\theta_3)*cos(\theta_2),                             -sin(\theta_2)*sin(\theta_3) + cos(\theta_2)*cos(\theta_3),              0,                                     a_2*sin(\theta_2) + a_3*sin(\theta_2)*cos(\theta_3) + a_3*sin(\theta_3)*cos(\theta_2) + d_1],
[                     

now we print all matrices

In [21]:
antropomorphic.print_transorm_matrices()

$$
^0T_1 = 
\left[\begin{matrix}\cos{\left (\theta_1 \right )} & 0 & \sin{\left (\theta_1 \right )} & 0\\\sin{\left (\theta_1 \right )} & 0 & - \cos{\left (\theta_1 \right )} & 0\\0 & 1 & 0 & d_{1}\\0 & 0 & 0 & 1\end{matrix}\right]
$$
$$
^1T_2 = 
\left[\begin{matrix}\cos{\left (\theta_2 \right )} & - \sin{\left (\theta_2 \right )} & 0 & a_{2} \cos{\left (\theta_2 \right )}\\\sin{\left (\theta_2 \right )} & \cos{\left (\theta_2 \right )} & 0 & a_{2} \sin{\left (\theta_2 \right )}\\0 & 0 & 1 & 0\\0 & 0 & 0 & 1\end{matrix}\right]
$$
$$
^2T_3 = 
\left[\begin{matrix}\cos{\left (\theta_3 \right )} & - \sin{\left (\theta_3 \right )} & 0 & a_{3} \cos{\left (\theta_3 \right )}\\\sin{\left (\theta_3 \right )} & \cos{\left (\theta_3 \right )} & 0 & a_{3} \sin{\left (\theta_3 \right )}\\0 & 0 & 1 & 0\\0 & 0 & 0 & 1\end{matrix}\right]
$$
$$
^0 T_3 = 
\left[\begin{matrix}- \sin{\left (\theta_2 \right )} \sin{\left (\theta_3 \right )} \cos{\left (\theta_1 \right )} + \cos{\left (\theta_1 \right )} \c

method `jacobian()` returns jacobian matrix. Arguments of this function are variables controlling a robot, that must be taken form `table` field (well, they don't actually, but the column of this not-a-variable would be zero).


In [22]:
j = antropomorphic.jacobian([antropomorphic.table[0]['t'], 
                         antropomorphic.table[1]['t'], 
                         antropomorphic.table[2]['t'] ])
print(r'$$')
print(str(sympy.latex( j )))
print(r'$$')
print('')

$$
\left[\begin{matrix}- \left(a_{2} \cos{\left (\theta_2 \right )} + a_{3} \cos{\left (\theta_2 + \theta_3 \right )}\right) \sin{\left (\theta_1 \right )} & - \left(a_{2} \sin{\left (\theta_2 \right )} + a_{3} \sin{\left (\theta_2 + \theta_3 \right )}\right) \cos{\left (\theta_1 \right )} & - a_{3} \sin{\left (\theta_2 + \theta_3 \right )} \cos{\left (\theta_1 \right )}\\\left(a_{2} \cos{\left (\theta_2 \right )} + a_{3} \cos{\left (\theta_2 + \theta_3 \right )}\right) \cos{\left (\theta_1 \right )} & - \left(a_{2} \sin{\left (\theta_2 \right )} + a_{3} \sin{\left (\theta_2 + \theta_3 \right )}\right) \sin{\left (\theta_1 \right )} & - a_{3} \sin{\left (\theta_1 \right )} \sin{\left (\theta_2 + \theta_3 \right )}\\0 & a_{2} \cos{\left (\theta_2 \right )} + a_{3} \cos{\left (\theta_2 + \theta_3 \right )} & a_{3} \cos{\left (\theta_2 + \theta_3 \right )}\end{matrix}\right]
$$

